In [2]:
import telepot
import logging
import os
import requests
from bs4 import BeautifulSoup

# log
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

TELEGRAM_TOKEN = 'TELEGRAM_TOKEN'

In [ ]:
# 명령어 디렉토리 가져오기
def get_dir_list(dir):
    str_list = ''
    if os.path.exists(dir):
        file_list = os.listdir(dir)
        file_list.sort()
        for f in file_list:
            full_path = os.path.join(dir, f)
            if os.path.isdir(full_path):
                f = '[' + f + ']'
            str_list += f
            str_list += '\n'
    str_list.strip()
    return str_list

In [4]:
# 날씨 정보 가져오기
def get_weather(location):
    weather = ''
    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_sug.asiw&fbm=0&acr=1&acq=tjdnfs&qdt=0&ie=utf8&query={location}+날씨'
    r = requests.get(url)
    bs = BeautifulSoup(r.text, 'lxml')
    w_box = bs.select('div.today_area > div.main_info')
    
    if len(w_box) > 0:
        temperature = bs.select('span.todaytemp')
        cast_text = bs.select('p.cast_txt')
        indicator = bs.select('span.indicator')
        
        if len(temperature) > 0 and len(cast_text) > 0 and len(indicator) > 0:
            temperature = temperature[0].text.strip()
            indicator = indicator[0].text.strip()
            txt = cast_text[0].text.strip()
            # Telegram에서는 캐리지 리턴 \r을 붙여줘야 한다
            weather = f'{temperature}℃\r\n{indicator}\r\n{txt}'
        return weather

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): search.naver.com:443
DEBUG:urllib3.connectionpool:https://search.naver.com:443 "GET /search.naver?where=nexearch&sm=top_sug.asiw&fbm=0&acr=1&acq=tjdnfs&qdt=0&ie=utf8&query=%EC%84%9C%EC%9A%B8+%EB%82%A0%EC%94%A8 HTTP/1.1" 200 None


15 자외선 3보통 맑음, 어제보다 1˚ 높아요


In [8]:
# 환율 정보 가져오기
MONEY_NAME = {
    "유로": "유럽연합 EUR",
    "엔": "일본 JPY (100엔)",
    "위안": "중국 CNY",
    "홍콩달라": "홍콩 HKD",
    "타이완달라": "대만 TWD",
    "파운드": "영국 GBP",
    # '달라'가 위 홍콩달라, 타이완달라에도 쓰이기 때문에 index 검색에서 달라가 먼저 걸리는것을 피하기 위해
    # 달라를 가장 마지막에 둔다
     "달라": "미국 USD",
}

def get_exchange_info():
    EXCHANGE_LIST = {}
    url = 'https://finance.naver.com/marketindex/exchangeList.nhn'
    r = requests.get(url)
    bs = BeautifulSoup(r.text, 'lxml')
    trs = bs.select('table.tbl_exchange > tbody > tr')
    for tr in trs:
        tds = tr.select('td')
        name = tds[0].text.strip()
        value = tds[1].text.strip().replace(',', '')
        EXCHANGE_LIST[name] = value
    return EXCHANGE_LIST
    
    
def money_translate(keyword):
    EXCHANGE_LIST = get_exchange_info()
    keywords = []
    for m in MONEY_NAME.keys():
        if m in keyword:
            keywords.append(keyword[0:keyword.find(m)].strip())
            keywords.append(m)
            break
    
    if keywords[1] in MONEY_NAME:
        country = MONEY_NAME[keywords[1]]
        
        if country in EXCHANGE_LIST:
            money = float(EXCHANGE_LIST[country])
            if country == '일본 JPY (100엔)':
                money /= 100
            
            money = format(round(float(money) * float(keywords[0]), 3), ',')
            output = f'{money} 원'
            return output
        
#print(money_translate('100 달라'))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): finance.naver.com:443
DEBUG:urllib3.connectionpool:https://finance.naver.com:443 "GET /marketindex/exchangeList.nhn HTTP/1.1" 200 3638


14,271.0 원


In [1]:
# Bot Handler
def handler(msg):
    content_type, chat_Type, chat_id, msg_id = telepot.glance(msg, long=True)
    
    print(msg)
    
    # /dir c:\\test
    if content_type == 'text':
        str_message = msg['text']
        if str_message[0:1] == '/':
            atgs = str_message.split(' ')
            command = args[0]
            del args[0]
            
            # 지정 경로안의 모든 파일 및 폴더 목록 가져오기
            if command == '/dir' or command == '/목록':
                filepath = ' '.join(args)
                if filepath.strip() == '':
                    bot.sendMessage(chat_id, '/dir [대상폴더] 로 입력해주세요.')
                else:
                    filelist = get_dir_list(filepath)
                    bot.sendMessage(chat_id, filelist)
                    
            # 지정 지역의 날씨 정보 가져오기
            elif command == '/weather' or command == '/날씨':
                w = ' '.join(args)
                weather = get_weather(w)
                bot.sendMessage(chat_id, weather)
                
            # 지정 환율정보 가져오기
            elif command == '/money' or command == '/환율':
                m = ' '.join(args)
                exchange = money_translate(m)
                bot.sendMessage(chat_id, exchange)
                
            # 지정 파일 다운로드하기
            elif command[0:4] == '/get':
                filepath = ' '.join(args)
                if os.path.exist(filepath):
                    try:
                        if command == '/getfile':
                            bot.sendDocument(chat_id, open(filepath, 'rb'))
                        elif command == '/getimage':
                             bot.sendPhoto(chat_id, open(filepath, 'rb'))
                        elif command == '/getaudio
                             bot.sendAudio(chat_id, open(filepath, 'rb'))
                        elif command == '/getvideo':
                             bot.sendVideo(chat_id, open(filepath, 'rb'))
                    except Exception as e:
                        bot.sendMessage(chat_id, f'파일 전송 실패: {e}')
                else:
                    bot.sendMessage(chat_id, '파일이 존재하지 않습니다.')
                            
                            
                            
bot = telepot.Bot(TELEGRAM_TOKEN)
bot.message_loop(handler, run_forever=True)

IndentationError: expected an indented block (<ipython-input-1-f988d30ec4f6>, line 18)